In [11]:
import os
import numpy as np
import tensorflow as tf
from utils.logger import Logger
import models.model_picker as mp

In [14]:
model_name = "conv_gdn_autoencoder_dropout"
model_version = "1.0"
model_log_file = "/home/dpaiton/Work/Projects/"+model_name+"/logfiles/"+model_name+"_v"+model_version+".log"

model_logger = Logger(model_log_file, overwrite=False)
model_log_text = model_logger.load_file()
model_params = model_logger.read_params(model_log_text)[-1]

model_schedule = model_logger.read_schedule(model_log_text)

In [15]:
num_logged_input_channels = len(model_params.input_channels)
model_params.input_channels = model_params.input_channels[:num_logged_input_channels//2]
num_logged_output_channels = len(model_params.output_channels)
model_params.output_channels = model_params.output_channels[:num_logged_output_channels//2]
num_logged_strides = len(model_params.strides)
model_params.strides = model_params.strides[:num_logged_strides//2]

In [16]:
model_params.model_dir = (os.path.expanduser("~")+"/Work/Projects/"+model_params.model_name)

analysis_out_dir = model_params.model_dir+"/analysis/"+model_params.version+"/"
if not os.path.exists(analysis_out_dir):
  os.makedirs(analysis_out_dir)
analysis_log_file = analysis_out_dir+"analysis.log"
model_params.model_out_dir = analysis_out_dir
model_params.cp_load = True
model_params.cp_load_name = model_name
model_params.cp_load_step = None
model_params.cp_load_ver = "1.0"
model_params.cp_load_dir = model_params.model_dir+"/checkpoints/"

In [17]:
model_params.gdn_w_init_const = 0.1
model_params.gdn_b_init_const = 0.1
model_params.gdn_w_thresh_min = 1e-6
model_params.gdn_b_thresh_min = 1e-6
model_params.gdn_eps = 1e-6

In [18]:
model = mp.get_model(model_params.model_type)
#model.w_init_list = [None,]*num_layers/2+[np.loadz("path_to_w"+idx) for idx in range(3,6)]
#...
model.setup(model_params)

/home/dpaiton/Work/DeepSparseCoding/utils/get_data.py:55: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Rs_min = Rs[min_Vs_indx]
/home/dpaiton/Work/DeepSparseCoding/utils/get_data.py:56: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
  Rs_max = Rs[max_Vs_indx]
/home/dpaiton/Work/DeepSparseCoding/utils/get_data.py:57: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will resul

In [19]:
feed_dict = model.get_feed_dict(np.zeros([model_params.batch_size]+model_params.data_shape, dtype=np.float32))
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
with tf.compat.v1.Session(config=config, graph=model.graph) as sess:
  sess.run(model.init_op, feed_dict)
  cp_load_file = tf.train.latest_checkpoint(model.cp_load_dir, model.cp_latest_filename)
  model.load_weights(sess, cp_load_file)
  tensors = [model.w_list, model.w_gdn_list, model.b_list, model.b_gdn_list]
  eval_list = sess.run(tensors, feed_dict)

INFO:tensorflow:Restoring parameters from /home/dpaiton/Work/Projects/conv_gdn_autoencoder_dropout/checkpoints/conv_gdn_autoencoder_dropout_v0.0_weights-100000


In [20]:
num_weights = len(eval_list[0])
half_weight_idx = int((num_weights/2))
decode_weights_list = eval_list[0][half_weight_idx:]
for decode_idx, decode_weights in enumerate(decode_weights_list):
  w_idx = str(decode_idx+half_weight_idx)
  print("weight ",w_idx," ",decode_weights.shape)
  np.savez(analysis_out_dir+"w"+w_idx+".npz", data=decode_weights)
  
decode_gdn_list = eval_list[1][half_weight_idx:]
for decode_idx, decode_gdn in enumerate(decode_gdn_list):
  w_idx = str(decode_idx+half_weight_idx)
  print("gdn ",w_idx," ",decode_gdn.shape)
  np.savez(analysis_out_dir+"w_gdn"+w_idx+".npz", data=decode_gdn)

decode_b_list = eval_list[2][half_weight_idx:]
for decode_idx, decode_b in enumerate(decode_b_list):
  w_idx = str(decode_idx+half_weight_idx)
  print("b ",w_idx," ",decode_b.shape)
  np.savez(analysis_out_dir+"b"+w_idx+".npz", data=decode_b)
  
decode_b_gdn_list = eval_list[3][half_weight_idx:]
for decode_idx, decode_b_gdn in enumerate(decode_b_gdn_list):
  w_idx = str(decode_idx+half_weight_idx)
  print("b_gdn", w_idx, " ", decode_b_gdn.shape)
  np.savez(analysis_out_dir+"b_gdn"+w_idx+".npz", data=decode_b_gdn)

weight  3   (4, 4, 64, 28)
weight  4   (9, 9, 128, 64)
weight  5   (8, 8, 1, 128)
gdn  3   (64, 64)
gdn  4   (128, 128)
b  3   (64,)
b  4   (128,)
b  5   (1,)
b_gdn 3   (64,)
b_gdn 4   (128,)
